In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

class ImageProcessorApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Image Processor")
        self.geometry("900x900") 

        # Load the image
        try:
            background_image = Image.open("123.png")  # Replace with your image file path
            self.background_photo = ImageTk.PhotoImage(background_image)  # Store persistently
        except Exception as e:
            print(f"Error loading background image: {e}")
            self.background_photo = None

        # Create a label widget to display the image
        if self.background_photo:
            background_label = tk.Label(self, image=self.background_photo)
            background_label.place(relwidth=1, relheight=1)  # Make the image fill the entire window

        # Additional widgets
        self.image_label = tk.Label(self)
        self.image_label.pack()
        self.image_label2 = tk.Label(self)
        self.image_label2.pack()
        

        # Frame for multiple selection buttons
        self.selection_frame = tk.Frame(self)
        self.selection_frame.pack(expand=True)
        self.selection_frame.place(relwidth=1)
        self.add_selection_buttons()

        # Frame for dynamic option buttons
        self.options_frame = tk.Frame(self)
        self.options_frame.pack(expand=True)
        
        self.image_label3 = tk.Label(self)
        self.image_label3.pack()

        self.image = None
        self.image2=None

    def add_selection_buttons(self):
        """Add multiple selection buttons in one row."""
        selections = [
            ("Image", lambda: self.display_options("Image")),
            ("Hist_Segmentaion", lambda: self.display_options("Histogram_Based_Segmentaion")),
            ("Halftone", lambda: self.display_options("Halftone")),
            ("Image_Operations", lambda: self.display_options("Image_Operations")),
            ("Histogram", lambda: self.display_options("Histogram")),
            ("Simple_Edge", lambda: self.display_options("Simple_Edge")),
            ("Advanced_Edge", lambda: self.display_options("Advanced_Edge")),
            ("Filtering", lambda: self.display_options("Filtering")),
            
        ]
        for text, command in selections:
            button = tk.Button(
                self.selection_frame, 
                # bg="blond",
                # fg="green",  
                bd=0,  
                text=text, font=("Helvetica", 14), 
                command=command, width=15)
            button.pack(side=tk.LEFT, padx=2)


    def display_options(self, selection_name):
        """Display options dynamically for the selected button."""
        # Clear existing options
        for widget in self.options_frame.winfo_children():
            widget.destroy()

        # Define options for the selected button
        if selection_name == "Halftone":
            options = [
                ("Simple", lambda: self.simple_halftone()),
                ("Advanced", lambda: self.Advanced_halftone()),
            ]

        elif selection_name == "Histogram":
            options = [
                ("Histogram", lambda: self.histogram()),
                ("Histogram_Equalization", lambda: self.histogram_equalization()),
            ]
        
        elif selection_name == "Simple_Edge":
            options = [
                ("Sobel", lambda: self.sobel_operator()),
                ("Prewitt", lambda: self.prewitt_operator()),
                ("Kirsch", lambda: self.kirsch_operator()),
            ]

        elif selection_name == "Advanced_Edge":
            options = [
                ("Homogeneity", lambda: self.homogeneity_simple_threshold()),
                ("difference_operator", lambda: self.difference_operator()),
                #("DOG_mask(7x7)", lambda: self.apply_mask77()),
               # ("DOG_mask(9x9)", lambda: self.apply_mask99()),
                ("DOG_mask", lambda: self.DOG_mask()),
                ("Contrast_Edge", lambda: self.edge()),
                ("Variance", lambda: self.compute_variance()),
                ("Range", lambda: self.compute_range()),
           ]
        
        elif selection_name == "Filtering":
            options = [
                ("High_Pass_Filter", lambda: self.high_pass_filter()),
                ("Low_Pass_Filter", lambda: self.low_pass_filter()),
                ("Median_Filter", lambda: self.median_filter()),
            ]

        elif selection_name == "Image_Operations":
            options = [
                ("Add", lambda: self.add()),
                ("Subtract", lambda: self.subtract()),
                ("Invert", lambda: self.invert()),
                ("Cut_Paste", lambda: self.cut_paste()),
            ]
            
        elif selection_name == "Histogram_Based_Segmentaion":
            options = [
                ("Manual", lambda: self.manual_segmentation()),
                ("Histogram_Peak", lambda: self.histogram_peak_segmentation()),
                ("Histogram_Valley", lambda: self.histogram_valley_segmentation()),
                ("Adaptive_Segmentation", lambda: self.adaptive_segmentation()),
            ]
        elif selection_name == "Image":
            options = [
                ("Upload_Image1", lambda: self.upload_image()),
                ("Upload_Image2", lambda: self.upload_image2()),
                ("Image_Gray", lambda: self.image_gray()),
                ("Threshold", lambda: self.threshold()),
            ]
        else:
            pass

        # Create option buttons dynamically
        for text, command in options:
            option_button = tk.Button(self.options_frame, text=text, font=("Helvetica", 12), command=command)
            option_button.pack(pady=5)


    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = cv2.imread(file_path)
            self.image = cv2.resize(self.image, (300, 300))
            self.image2 = cv2.imread(file_path)
            self.image2 = cv2.resize(self.image, (300, 300))
            self.display_image3(self.image)

    def upload_image2(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image2 = cv2.imread(file_path)
            self.image2 = cv2.resize(self.image2, (300, 300))
            self.display_image2(self.image2)
    
            
    def display_image(self, img):
        if img.dtype == np.float32:
            img = np.clip(img, 0, 255)  # Ensure values are within [0, 255]
            img = img.astype(np.uint8)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        self.image_label.config(image=img_tk)
        self.image_label.image = img_tk
        self.image_label.pack(side=tk.RIGHT, padx=10) 
        # الصورة الأولى على العمود الأول
    def display_image2(self, img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        self.image_label2.config(image=img_tk)
        self.image_label2.image = img_tk
        self.image_label2.pack(side=tk.LEFT, padx=20)  
        
    def display_image3(self, img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        self.image_label3.config(image=img_tk)
        self.image_label3.image = img_tk
        self.image_label3.pack(side=tk.LEFT, padx=30)
    
     
#############################          (Gray Scale)               ###################################  
    def image_gray(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        gray_image = self.do_gray(self.image)
        self.display_image(gray_image)
    
    def do_gray(self, image):
        if len(image.shape) == 3:  # Image has 3 channels (color)
            # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            r, g, b = image[:,:,0], image[:,:,1], image[:,:,2]
            gray_image = (0.299 * r + 0.587 * g + 0.114 * b).astype(np.uint8)
        else:  # Image is already grayscale
            gray_image = image
        return gray_image

####################################################################################################
#############################          (Threshold)               ###################################  
    def threshold(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        value=self.do_threshold(self.image)
        messagebox.showerror("The Threshold",int(value))
    
    
    def do_threshold(self, image): # Return value of threshold
        grayimg=self.do_gray(image)
        total_sum = 0
        pixel_count = 0
        img = np.array(grayimg)
        for row in img:
            for pixel in row:
                total_sum += pixel
                pixel_count += 1 
        threshold = total_sum / pixel_count
        return threshold
    
    def apply_threshold(self, image, threshold): # Return image after thresholding
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        grayimage = self.do_gray(image)
        binary_array = np.array(grayimage)
        for row in range(binary_array.shape[0]):
            for col in range(binary_array.shape[1]):
                if binary_array[row,col] < threshold:
                    binary_array[row,col] = 0
                else:
                    binary_array[row,col] = 255 
        return binary_array
    
    def calculate_threshold(self, peaks_indices):
        peak1 = peaks_indices[0]
        peak2 = peaks_indices[1]
        low_threshold = (peak1 + peak2) / 2  # Midpoint
        high_threshold = peak2  # Set second peak as high threshold (object)
        return low_threshold, high_threshold
    
    def edge_threshold(self, image, threshold):
        height, width = len(image), len(image[0])
        binary = [[0 for _ in range(width)] for _ in range(height)]
        
        for i in range(height):
            for j in range(width):
                binary[i][j] = 255 if image[i][j] > threshold else 0
                
        binary_array = np.array(binary, dtype=np.uint8)
        return binary_array
    
###################################################################################################
#############################          (Halftone)               ###################################  
    def simple_halftone(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        halftone_image = self.do_halftone(self.image)
        self.display_image(halftone_image)

    def do_halftone(self, image): 
        threshold = self.do_threshold(image)      
        halftone = self.apply_threshold(image, threshold) 
        return halftone

    ###################################
    def Advanced_halftone (self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        threshold = self.do_threshold(self.image)
        grayimage=self.do_gray(self.image)
        img_array = np.array(grayimage)
        halftone_array = np.zeros_like(img_array)
        for row in range(img_array.shape[0]):
            for col in range(img_array.shape[1]):
                old_pixel = img_array[row, col]
                new_pixel = 255 if old_pixel > threshold else 0  
                halftone_array[row, col] = new_pixel
                error = old_pixel - new_pixel
                if col + 1 < img_array.shape[1]:
                    img_array[row, col + 1] += error * 7 / 16
                if col - 1 >= 0 and row + 1 < img_array.shape[0]:
                    img_array[row + 1, col - 1] += error * 3 / 16
                if row + 1 < img_array.shape[0]:
                    img_array[row + 1, col] += error * 5 / 16
                if col + 1 < img_array.shape[1] and row + 1 < img_array.shape[0]:
                    img_array[row + 1, col + 1] += error * 1 / 16
        
        self.display_image(halftone_array)

####################################################################################################
#############################          (Histogram)               ###################################  
    def histogram (self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        histogram=self.do_histogram(self.image)
        
        
        hist_img = np.zeros((300, 256), dtype=np.uint8) 
        max_value = max(histogram)
        for x in range(256):
            bar_height = int((histogram[x] / max_value) * 300) if max_value > 0 else 0
            for y in range(bar_height):
                hist_img[300 - 1 - y, x] = 255  
        self.display_image(hist_img)
        
        
    def do_histogram(self,image):
        grayimage=self.do_gray(image)
        arrayimage= np.array(grayimage)
        histogram = np.zeros(256, dtype=int)
        for x in range(arrayimage.shape[0]):
            for y in range(arrayimage.shape[1]):
                value = arrayimage[x, y]
                histogram[value] += 1 
        return histogram
    #####################################


    def cumlative(self,hist):
        sum_of_hist = np.zeros(256, dtype=int)  # Array to store cumulative histogram
        running_sum = 0  # Running sum for cumulative distribution
        for i in range(256):
            running_sum += hist[i]
            sum_of_hist[i] = running_sum  # Ensure scalar assignment
        return sum_of_hist
    

    def histogram_equalization (self):
        gray_image=self.do_gray(self.image)
        hist = self.do_histogram(gray_image)  # Calculate histogram
        sum_of_hist = self.cumlative(hist)  # Calculate cumulative histogram
        total_pixels = gray_image.size  # Total number of pixels

    # Normalize the cumulative histogram to [0, 255]
        norm_cdf = (sum_of_hist * 255 / total_pixels).astype(np.uint8)

    # Map original pixels to equalized values
        equalized_image = np.zeros_like(gray_image)
        for i in range(gray_image.shape[0]):
            for j in range(gray_image.shape[1]):
                equalized_image[i, j] = norm_cdf[gray_image[i, j]]
        self.display_image(equalized_image)
        

########################################################################################################
#######################        (Simple Edge Detection)             #####################################

    def add_padding(self, image, pad_width=1):
        """Adds padding to the image."""
        padded_image = np.zeros((image.shape[0] + 2 * pad_width, image.shape[1] + 2 * pad_width))
        padded_image[pad_width:-pad_width, pad_width:-pad_width] = image
        return padded_image
    
    def convolution(self, image, kernel):
        """Performs convolution of an image with a given kernel."""
        kernel_height, kernel_width = kernel.shape
        image_height, image_width = image.shape
        output = np.zeros_like(image)

        for i in range(1, image_height - 1):
            for j in range(1, image_width - 1):
                region = image[i - 1:i + 2, j - 1:j + 2]
                output[i, j] = np.sum(region * kernel)
        return output

    @staticmethod
    def sobel_kernels():
        """Returns the Sobel kernels for edge detection."""
        Gx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        Gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
        return Gx, Gy

    def sobel_operator(self):
        """Applies the Sobel operator to detect edges in the image."""
        img_gray = self.do_gray(self.image)  # Assuming do_gray is defined elsewhere
        Gx, Gy = self.sobel_kernels()
        padded_image = self.add_padding(img_gray)

        gradient_x = self.convolution(padded_image, Gx)
        gradient_y = self.convolution(padded_image, Gy)

        gradient_magnitude = np.sqrt(np.square(gradient_x) + np.square(gradient_y))
        gradient_magnitude = np.clip(gradient_magnitude, 0, 255)
        return self.display_image(gradient_magnitude.astype(np.uint8))
    
        #####################################
    def prewitt_operator(self):
        image = self.do_gray(self.image)
        padding_image = self.add_padding(image)
        Gx = np.array([ [-1,0,1], [-1,0,1], [-1,0,1] ])
        Gy = np.array([ [1,1,1], [0,0,0], [-1,-1,-1] ])
        gradient_x = self.convolution(padding_image, Gx)
        gradient_y = self.convolution(padding_image, Gy)

        gradient_magnitude = np.sqrt(np.square(gradient_x) + np.square(gradient_y))
        gradient_magnitude = np.clip(gradient_magnitude, 0,255)
        return self.display_image(gradient_magnitude.astype(np.uint8))
    
    ########################################
    def kirsch_operator(self):
        image = self.do_gray(self.image)
        kernels = [
            np.array([[ 5,  5,  5], [-3,  0, -3], [-3, -3, -3]]),  # North
            np.array([[-3,  5,  5], [-3,  0,  5], [-3, -3, -3]]),  # North-East
            np.array([[-3, -3,  5], [-3,  0,  5], [-3, -3,  5]]),  # East
            np.array([[-3, -3, -3], [-3,  0,  5], [-3,  5,  5]]),  # South-East
            np.array([[-3, -3, -3], [-3,  0, -3], [ 5,  5,  5]]),  # South
            np.array([[-3, -3, -3], [ 5,  0, -3], [ 5,  5, -3]]),  # South-West
            np.array([[ 5, -3, -3], [ 5,  0, -3], [ 5, -3, -3]]),  # West
            np.array([[ 5,  5, -3], [ 5,  0, -3], [-3, -3, -3]])   # North-West
        ]
        padding_image = self.add_padding(image)
        gradients = np.zeros((len(kernels), image.shape[0], image.shape[1]))


        for k, kernel in enumerate(kernels):
            convolved = self.convolution(padding_image, kernel)
            gradients[k] =convolved[:image.shape[0], :image.shape[1]]

        
        edge_magnitude = np.max(gradients, axis=0)
        edge_magnitude = np.clip(edge_magnitude, 0, 255)
        
        
        edge_direction = np.argmax(gradients, axis=0)
        
        img = cv2.normalize(edge_direction, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        
        self.display_image(img)
        self.display_image2(edge_magnitude.astype(np.uint8))
        
        
########################################################################################################
#######################        (Advanced Edge Detection)             #####################################  
  
    def homogeneity_simple_threshold(self):
        image=self.do_gray(self.image)
        threshold = self.do_threshold(self.image)
        thresholded_image = self.apply_threshold(image, threshold)
        return self.homogeneity_operator(thresholded_image,threshold)
    
    def homogeneity_operator(self, image,threshold):
        height, width = image.shape
        homogeneity_image = np.zeros_like(image, dtype=np.uint8)

        for i in range(1, height - 1):
            for j in range(1, width - 1):
                center_pixel = image[i, j]
            # Get the 8-connected neighbors
                neighbors = [
                    image[i-1, j-1],  # Top-left
                    image[i-1, j],    # Top
                    image[i-1, j+1],  # Top-right
                    image[i, j-1],    # Left
                    image[i, j+1],    # Right
                    image[i+1, j-1],  # Bottom-left
                    image[i+1, j],    # Bottom
                    image[i+1, j+1]   # Bottom-right
                ]
            # Calculate the maximum absolute difference
                max_diff = max(abs(center_pixel - neighbor) for neighbor in neighbors)
                homogeneity_image[i, j] = max_diff
                homogeneity_image[i, j]=np.where(homogeneity_image[i, j]>=threshold,homogeneity_image[i, j],0)
                
        return self.display_image(homogeneity_image)
    
        #################################################
    def difference_operator(self):
        image=self.do_gray(self.image)
        threshold = self.do_threshold(image)
        thresholded_image = self.apply_threshold(image, threshold)
        return self.diff_operator(thresholded_image,threshold)

    
    def diff_operator(self,image,mean_intensity):
        height, width = image.shape
        homogeneity_image = np.zeros_like(image, dtype=np.uint8)

        for i in range(1, height - 1):
            for j in range(1, width - 1):
                center_pixel = image[i, j]
            # Get the 8-connected neighbors
                neighbors = [
                    image[i-1, j-1]-image[i+1, j+1],  # Top-left
                    image[i-1, j]-image[i+1, j],    # Top
                    image[i-1, j+1]-image[i+1, j-1],  # Top-right
                    image[i, j-1]-image[i, j+1]   # Left
                
                ]
            # Calculate the maximum absolute difference
                max_diff = max(abs( neighbor) for neighbor in neighbors)
                homogeneity_image[i, j] = max_diff
                homogeneity_image[i, j]=np.where(homogeneity_image[i, j]>=mean_intensity,homogeneity_image[i, j],0)

        return self.display_image(homogeneity_image)
        ###################################################3
            
    def apply_mask77(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        image=self.do_gray(self.image)
        mask = np.array([
            [0, 0, -1, -1, -1, 0, 0],
            [0, -2, -3, -3, -3, -2, 0],
            [-1, -3, 5, 5, 5, -3, -1],
            [-1, -3, 5, 16, 5, -3, -1],
            [-1, -3, 5, 5, 5, -3, -1],
            [0, -2, -3, -3, -3, -2, 0],
            [0, 0, -1, -1, -1, 0, 0]
        ])
        rows, cols = image.shape
        output_image = np.zeros_like(image)

    
        pad_size = mask.shape[0] // 2
        padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant', constant_values=0)

    # Apply mask (convolution)
        for i in range(pad_size, rows + pad_size):
            for j in range(pad_size, cols + pad_size):
                region = padded_image[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
                result = np.sum(region * mask)
                output_image[i - pad_size, j - pad_size] = np.clip(result, 0, 255)  # Ensure values are within [0, 255]

        return output_image
    
    def apply_mask99(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        image=self.do_gray(self.image)
        mask = np.array([
            [0, 0, 0, -1, -1, -1, 0, 0, 0],
            [0, -2, -3, -3, -3, -3, -2, 0, 0],
            [-1, -3, -2, -1, -1, -1, -2, -3, 0],
            [-1, -3, -1, 9, 9, 9, -1, -3, -1],
            [-1, -3, -1, 9, 19, 9, -1, -3, -1],
            [-1, -3, -1, 9, 9, 9, -1, -3, -1],
            [-1, -3, -2, -1, -1, -1, -2, -3, 0],
            [0, -2, -3, -3, -3, -3, -2, 0, 0],
            [0, 0, 0, -1, -1, -1, 0, 0, 0]
        ])
        rows, cols = image.shape
        output_image = np.zeros_like(image)

    
        pad_size = mask.shape[0] // 2
        padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant', constant_values=0)

    # Apply mask (convolution)
        for i in range(pad_size, rows + pad_size):
            for j in range(pad_size, cols + pad_size):
                region = padded_image[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
                result = np.sum(region * mask)
                output_image[i - pad_size, j - pad_size] = np.clip(result, 0, 255)  # Ensure values are within [0, 255]

        return output_image
    
    
    
    def DOG_mask(self):
        image1=self.apply_mask77()
        image2=self.apply_mask99()
        return self.display_image(image1-image2)
    
#####################################3
    def edge(self):
        edge_mask = np.array([
            [-1, 0, -1],
            [0, 4, 0],
            [-1, 0, -1]
        ])

        smoothing_mask = np.ones((3,3)) / 9
        image=self.do_gray(self.image)
        edge_output = cv2.filter2D(image, -1, edge_mask)

        average_output = cv2.filter2D(image, -1, smoothing_mask)
        average_output = average_output.astype(float)
        average_output += 1e-10

        contrast_edges = (edge_output / average_output) * 255  # Scale to 0-255 range
        contrast_edges = contrast_edges.astype(np.uint8)

        # return contrast_edges, edge_output, average_output
        return self.display_image(contrast_edges)
        ####################################################
    def compute_variance(self, window_size=5):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        image =self.do_gray(self.image)
        height, width = image.shape
        half_window = window_size // 2
        variance = np.zeros_like(image)

        for i in range(half_window, height - half_window):
            for j in range(half_window, width - half_window):
                window = image[i-half_window:i+half_window+1, j-half_window:j+half_window+1]
                window_variance = np.var(window)
                variance[i, j] = window_variance

        return self.display_image(variance)
    ##############################################################
    def compute_range(self, window_size=5):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        image =self.do_gray(self.image)
        height, width = image.shape
        half_window = window_size // 2
        range_image = np.zeros_like(image)

        for i in range(half_window, height - half_window):
            for j in range(half_window, width - half_window):
                window = image[i-half_window:i+half_window+1, j-half_window:j+half_window+1]
                window_range = np.max(window) - np.min(window)
                range_image[i, j] = window_range

        return self.display_image(range_image)
        
########################################################################################################
#######################        (Filtering)             #####################################    


    def compute_gradient_magnitude(self, grad_x, grad_y):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        height, width = len(grad_x), len(grad_x[0])
        magnitude = [[0 for _ in range(width)] for _ in range(height)]
        for i in range(height):
            for j in range(width):
                magnitude[i][j] = (grad_x[i][j]**2 + grad_y[i][j]**2)**0.5
        return magnitude

    def normalize(self, image):
        height, width = len(image), len(image[0])
        max_val = max(max(row) for row in image)
        min_val = min(min(row) for row in image)
        normalized = [[0 for _ in range(width)] for _ in range(height)]
        
        for i in range(height):
            for j in range(width):
                normalized[i][j] = int(255 * (image[i][j] - min_val) / (max_val - min_val)) if max_val != min_val else 0
        return normalized

    ################################################################
    def low_pass_filter(self):#reduce noise and details, blur img
        #Checking if an image is loaded:
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
            #Converting the image to grayscale:
        gray_img = self.do_gray(self.image)
        #creates a smoothing effect by averaging the pixel intensities.
        mask_3x3_low_pass=np.array([
            [0,1/6,0],
            [1/6,2/6,1/6],
            [0,1/6,0]
        ], dtype=np.float32)
        #cv2.filter2d applies convolution
        result= cv2.filter2D(gray_img, -1 , mask_3x3_low_pass)# -1 --> output image will have the same depth
        self.display_image(result)
        ###########################################################

    def high_pass_filter(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        gray_img = self.do_gray(self.image)   
        mask_3x3_high_pass=np.array([
            [0,-1,0],
            [-1,5,-1],
            [0,-1,0]
        ], dtype=np.float32)
        result= cv2.filter2D(gray_img, -1 , mask_3x3_high_pass)
        self.display_image(result)
        ######################################################

    def median_filter(self):# replace the center point to median of the surrounding pixels , smooter img
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        gray_img = self.do_gray(self.image)
        #get dimentions of img
        height, width = gray_img.shape
        #output array
        filtered_image = np.zeros_like(gray_img)
        # loop to points accept border pixels (to avoid out-of-bound)
        for i in range(1,height -1):
            for j in range(1, width -1):
                #extract 3x3 block of pixel
                neightborhood = gray_img[i-1:i+2 , j-1:j+2]
                median_value=np.median(neightborhood)
                filtered_image[i,j] = median_value
                #noise reduction , edge prevention 
        self.display_image(filtered_image)

########################################################################################################
#######################        (Image Operation)             #####################################    
    def add(self):
    # Ensure the images have the same shape
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        if self.image2 is None:
            messagebox.showerror("Error", "No image uploaded!")
            return    
        gray_img = self.do_gray(self.image)
        gray_img2 = self.do_gray(self.image2)
        #get the dimentions
        height, width = gray_img.shape
        #unit8--> to ensure the pixel value stay in valid range[0,255]
        added_image = np.zeros((height, width), dtype=np.uint8)
        for i in range(height):
            for j in range(width):
                value =int( gray_img[i, j]) +int( gray_img2[i, j])
                added_image[i, j] = np.clip(value,0,255)  # Clipping to [0, 255]
        self.display_image(added_image)
     #####################################

    def subtract(self):
    # Ensure the images have the same shape
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        if self.image2 is None:
            messagebox.showerror("Error", "No image uploaded!")
            return    
        gray_img = self.do_gray(self.image)
        gray_img2 = self.do_gray(self.image2)
        height, width = gray_img.shape
        subtracted_image = np.zeros((height, width), dtype=np.uint8)
        for i in range(height):
            for j in range(width):
                value = gray_img[i, j] - gray_img2[i, j]
                subtracted_image[i, j] = max(0, min(int(value), 255))  # Clipping to [0, 255]
        self.display_image(subtracted_image) 
        ######################################################

    def invert(self):
    # Ensure the images have the same shape
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        gray_img = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        inverted_image = 255 - gray_img
        self.display_image(inverted_image)
           #######################################################


    def cut_paste(self):#useful for composite image creation and experimenting with image processing results
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        if self.image2 is None:
            messagebox.showerror("Error", "No image uploaded!")
            return    
        gray_img = self.do_gray(self.image)
        gray_img2 = self.do_gray(self.image2)
        x, y = 50,50 #position
        w, h = 100,100 #size
        h1, w1 = gray_img.shape
        h2, w2 = gray_img2.shape
        #validating the region
        if x + w > gray_img.shape[1] or y + h > gray_img.shape[0]:
            messagebox.showerror("Error", "Cut region exceeds image 1 dimensions!")
            return
        if x + w > gray_img2.shape[1] or y + h > gray_img2.shape[0]:
            messagebox.showerror("Error", "Paste region exceeds image 2 dimensions!")
            return

    # Cut the region from image1
        cut_image = gray_img[y:y+h, x:x+w]

    # Copy image2 and paste the cut region onto it
        output_image = np.copy(gray_img2)
        output_image[y:y+h, x:x+w] = cut_image

        self.display_image(output_image)
    


########################################################################################################
#######################        (Histogram Based Segmentation)             #####################################    
    def manual_segmentation(self):#Suitable for fine-tuning but time-consuming.
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return
        gray_img = self.do_gray(self.image)
        #define the thresholdind
        low_threshold = 50
        high_threshold = 150
        #applay thresolding
        segmented_img = np.where((gray_img >= low_threshold) & (gray_img <= high_threshold), 255, 0).astype(np.uint8)
        self.display_image(segmented_img)

    ###############################################  
    #1-compute histogram, 2-find peaks,3-sort peaks,4-select background , object , 5-cal threshold,6-apply threshold
    #choose the midpoint
    
    def find_histogram_peaks(self, hist):
        peaks,_ = find_peaks(hist.flatten())  # find_peaks function
        # Sort peaks by the height of the histogram

        sorted_peaks = sorted(peaks, key=lambda x: hist[x], reverse=True)
        return sorted(sorted_peaks[:2])  # Keep the highest 2 peaks

    def histogram_peak_segmentation(self):
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return

        # Convert image to grayscale
        gray_img =self.do_gray(self.image)

        # Calculate the histogram
        hist = self.do_histogram(gray_img)

        # Find peaks in the histogram
        peaks_indices = self.find_histogram_peaks(hist)

        # Calculate the low and high thresholds based on the peaks
        low_threshold, high_threshold = self.calculate_threshold(peaks_indices)

        # Create a segmented image
        segmented_image = np.zeros_like(gray_img)
        #segmented_image[(gray_img >= low_threshold) & (gray_img <= high_threshold)] = 255  # Set to white (255)
        segmented_image = np.where((gray_img >= low_threshold) & (gray_img <= high_threshold), 255, 0).astype(np.uint8)
        # Display the segmented image (implement this method to display the result)
        self.display_image(segmented_image)
    #############################################

    #1-calculate histo,2-detect peaks,3-sort peaks,4-find the point,5-segment the img
    #choose the lowestpoint
    def histogram_valley_segmentation(self):
         # Step 1: Ensure an image is loaded
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return

    # Step 2: Convert the image to grayscale
        gray_img = self.do_gray(self.image)

    # Step 3: Calculate the histogram
        hist =self.do_histogram(gray_img)
        hist = hist.flatten()
        if hist is None or len(hist) == 0:
            messagebox.showerror("Error", "Failed to compute histogram!")
            return



    # Step 4: Detect peaks in the histogram
        peaks,_ = find_peaks(hist)
        if len(peaks) < 2:
            messagebox.showerror("Error", "Not enough peaks found!")
            return

    # Step 5: Sort peaks and find the valley point
        sorted_peaks = sorted(peaks, key=lambda x: hist[x], reverse=True)
        peak1, peak2 = sorted(sorted_peaks[:2])  # Take the two most prominent peaks
        valley = np.argmin(hist[peak1:peak2]) + peak1  # Find the lowest point (valley) between peaks

    # Segment the image based on the valley
        segmented_img = np.where(gray_img <= valley, 0, 255).astype(np.uint8)

    # Display the segmented image

        self.display_image(segmented_img)
    ##################################################

        #1-calculate histo,2-detect peaks,3-sort peaks,4-findthe vally point,5-first-pass segmentation,6-calculate mean, 
    #7-adjust the threshold, 8-second-pass segmentation
    from scipy.signal import find_peaks
    def adaptive_segmentation(self):
    # Step 1: Ensure an image is loaded
        if self.image is None:
            messagebox.showerror("Error", "No image uploaded!")
            return

    # Step 2: Convert the image to grayscale
        gray_img = self.do_gray(self.image)

    # Step 3: Calculate the histogram
        hist =self.do_histogram(gray_img)
        hist = hist.flatten()
        if hist is None or len(hist) == 0:
            messagebox.showerror("Error", "Failed to compute histogram!")
            return

    # Step 4: Detect peaks in the histogram
        peaks, _ =find_peaks(hist)
        if len(peaks) < 2:
            messagebox.showerror("Error", "Not enough peaks found!")
            return

    # Step 5: Sort peaks by intensity value and find the valley point
        sorted_peaks = sorted(peaks, key=lambda x: hist[x], reverse=True)
        peak1, peak2 = sorted(sorted_peaks[:2])  # Take the two most prominent peaks
        valley = np.argmin(hist[peak1:peak2]) + peak1  # Index of the minimum value between the peaks

    # First-pass segmentation
        segmented_img = np.where(
        (gray_img >= peak1) & (gray_img <= valley), 255, 0
         ).astype(np.uint8)

    # Step 6: Calculate mean intensity for object and background pixels
        object_pixels = gray_img[segmented_img == 255]
        background_pixels = gray_img[segmented_img == 0]

        object_mean = object_pixels.mean() if object_pixels.size > 0 else 0
        background_mean = background_pixels.mean() if background_pixels.size > 0 else 0

    # Step 7: Adjust the threshold using calculated means
        new_low_threshold, new_high_threshold = sorted([object_mean, background_mean])

    # Step 8: Second-pass segmentation
        final_segmented_img = np.where(
        (gray_img >= new_low_threshold) & (gray_img <= new_high_threshold), 255, 0
    ).astype(np.uint8)

    # Display the final segmented image
        try:
            self.display_image(final_segmented_img)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to display image: {str(e)}")   
        self.display_image(final_segmented_img)    

    ###########################################################

        




if __name__ == "__main__":
    app = ImageProcessorApp()
    app.mainloop()

Error loading background image: [Errno 2] No such file or directory: 'C:\\Users\\MR.GENIUS\\Desktop\\image_pro\\123.png'


C:\Users\MR.GENIUS\AppData\Local\Temp\ipykernel_4164\1203962897.py:481: RuntimeWarning: overflow encountered in scalar subtract
  max_diff = max(abs(center_pixel - neighbor) for neighbor in neighbors)
C:\Users\MR.GENIUS\AppData\Local\Temp\ipykernel_4164\1203962897.py:504: RuntimeWarning: overflow encountered in scalar subtract
  image[i-1, j-1]-image[i+1, j+1],  # Top-left
C:\Users\MR.GENIUS\AppData\Local\Temp\ipykernel_4164\1203962897.py:505: RuntimeWarning: overflow encountered in scalar subtract
  image[i-1, j]-image[i+1, j],    # Top
C:\Users\MR.GENIUS\AppData\Local\Temp\ipykernel_4164\1203962897.py:506: RuntimeWarning: overflow encountered in scalar subtract
  image[i-1, j+1]-image[i+1, j-1],  # Top-right
C:\Users\MR.GENIUS\AppData\Local\Temp\ipykernel_4164\1203962897.py:507: RuntimeWarning: overflow encountered in scalar subtract
  image[i, j-1]-image[i, j+1]   # Left
